In [8]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

In [9]:
def busca_impi(medicamento):
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        tabla=driver.find_element(By.CLASS_NAME, "ms-listviewtable")
        tabla_content=tabla.get_attribute('outerHTML')
        return tabla_content
        #Código para extraer los datos de la página
    url = "https://patmedsp.impi.gob.mx/Lists/MedicamentosPat/MIO.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    time.sleep(5)
    cuadro_busqueda = driver.find_element(By.ID, "inplaceSearchDiv_WPQ1_lsinput")
    cuadro_busqueda.send_keys(medicamento)
    #Dar enter
    cuadro_busqueda.send_keys(u'\ue007')
    #Obtener el html
    time.sleep(5)
    try:
        table_contents=elementos_tabla(driver)
    except NoSuchElementException:
        driver.quit()
        return pd.DataFrame()
    
    #Obtener el html
    soup = BeautifulSoup(table_contents, 'html.parser')
    renglones = soup.find_all('tr')
    # #Obtener columnas de renglones
    columnas = [tr.find_all('td') for tr in renglones]
    #Dejar solo texto de columnas
    columnas = [[ele.text.strip() for ele in col] for col in columnas]
    
    #Crear dataframe
    df = pd.DataFrame(columnas)
    if df.empty:
        driver.quit()
        #Regresar dataframe con columnas vacías
        return pd.DataFrame()
    
    #Limpieza de dataframe
    df = df.iloc[1:]
    #Eliminar primera columna
    df = df.iloc[:,1:]
    #Renombrar columnas
    df.columns = ['id_ficha', 'num_concesion', 'titular', 'nom_generico', 'desc_esp', 'vigencia', 'anualidades', 'reivindicacines', 'obs', 
              'tipo_patente', 'enlace_med', 'ano_vigente', 'ind_terap', 'naturaleza','modif','creado']
    #Eliminar espacios en blanco de todas las columnas
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    #Eliminar \n de todas las columnas
    df = df.apply(lambda x: x.str.replace('\n', ' ') if x.dtype == "object" else x)
    #Minúsculas todas las columnas
    df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
    #Eliminar ñ y acentos de todas las columnas
    df = df.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') if x.dtype == "object" else x)
    driver.quit()
    return df    

In [10]:
busca_impi("sorafenib")

,id_ficha,num_concesion,titular,nom_generico,desc_esp,vigencia,anualidades,reivindicacines,obs,tipo_patente,enlace_med,ano_vigente,ind_terap,naturaleza,modif,creado
1,303,284193,bayer healthcare llc,sorafenib,tosilato de sorafenib,22-febrero-2026,pago cubierto hasta el final de la vigencia,reivindicacion 1. una composicion farmaceutica...,tipo de patente: composicion farmaceutica l...,composicion farmaceutica,mx/t/2011/25980,2026,"cancer de mama, del aparato respiratorio, cere...",,22 de febrero,mario iran florentino parra hernandez
2,769,238942,bayer healthcare llc,sorafenib,,ver observaciones,ver observaciones,reivindicacion 1. compuesto seleccionado del g...,tipo de patente: principio activo. licencia de...,principio activo,mx/s/2007/019384,2023,,,23 de febrero,karina aurora gonzalez segura
